## Iteration 1

The first iteration is just a barebones skeleton of the future solution. It contains a model (gpt-4o for sanity check) and a basic system prompt that asks the model to generate a task in Ukrainian without providing any details or criteria, and compares the model's output with a user-created task.

The dataset has just three fields (task, solution, and answer) and only one task type: open questions that require an answer.

In [1]:
import jsonlines
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
load_dotenv()

True

In [4]:
with jsonlines.open('dataset.jsonl') as reader:
    dataset = list(reader)

dataset[0]

{'topic': 'Арифметичні дії',
 'task': 'Обчисліть значення виразу: $100 - (15 + 5) \\times 4$.',
 'solution': 'Спочатку виконуємо дію в дужках: $15 + 5 = 20$. Далі виконуємо множення: $20 \\times 4 = 80$. Наостанок виконуємо віднімання: $100 - 80 = 20$.',
 'answer': '20'}

In [5]:
system_prompt = """Ти - вчитель математики. Створи одну математичну задачу на задану тему та надай її рішення та фінальну відповідь.
Формат відповіді:
ЗАДАЧА: [текст задачі]
РІШЕННЯ: [хід рішення]
ВІДПОВІДЬ: [фінальна відповідь]"""

In [6]:
model = ChatOpenAI(model="gpt-4o", temperature=0.1)

In [7]:
for item in dataset[:5]:

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=item['topic'])
    ]

    response = model.invoke(messages)
    
    display(pd.DataFrame([
        {'field': 'topic', 'value': item['topic']},
        {'field': 'sample_task', 'value': item['task']},
        {'field': 'sample_solution', 'value': item['solution']},
        {'field': 'sample_answer', 'value': item['answer']},
        {'field': 'system_prompt', 'value': messages[0].content},
        {'field': 'user_message', 'value': messages[1].content},
        {'field': 'model_response', 'value': response.content},
    ]))

,field,value
0,topic,Арифметичні дії
1,sample_task,Обчисліть значення виразу: $100 - (15 + 5) \times 4$.
2,sample_solution,Спочатку виконуємо дію в дужках: $15 + 5 = 20$. Далі виконуємо множення: $20 \times 4 = 80$. Наостанок виконуємо віднімання: $100 - 80 = 20$.
3,sample_answer,20
4,system_prompt,Ти - вчитель математики. Створи одну математичну задачу на задану тему та надай її рішення та фінальну відповідь.\nФормат відповіді:\nЗАДАЧА: [текст задачі]\nРІШЕННЯ: [хід рішення]\nВІДПОВІДЬ: [фінальна відповідь]
5,user_message,Арифметичні дії
6,model_response,"ЗАДАЧА: У класі є 24 учні. Вчителька вирішила розділити їх на групи так, щоб у кожній групі було по 4 учні. Скільки груп вийде?\n\nРІШЕННЯ: Щоб знайти кількість груп, потрібно загальну кількість учнів поділити на кількість учнів у кожній групі. Отже, ділимо 24 на 4:\n\n24 ÷ 4 = 6\n\nТаким чином, вийде 6 груп.\n\nВІДПОВІДЬ: 6 груп."


,field,value
0,topic,Спрощення виразів
1,sample_task,"Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."
2,sample_solution,"Використовуємо формулу скороченого множення $(a - b)(a + b) = a^2 - b^2$. У даному випадку $a = x$, а $b = 3$. Тому $(x - 3)(x + 3) = x^2 - 3^2 = x^2 - 9$."
3,sample_answer,$x^2 - 9$
4,system_prompt,Ти - вчитель математики. Створи одну математичну задачу на задану тему та надай її рішення та фінальну відповідь.\nФормат відповіді:\nЗАДАЧА: [текст задачі]\nРІШЕННЯ: [хід рішення]\nВІДПОВІДЬ: [фінальна відповідь]
5,user_message,Спрощення виразів
6,model_response,"ЗАДАЧА: Спростіть вираз: \( 3x^2 - 5x + 7 - 2x^2 + 4x - 9 \).\n\nРІШЕННЯ: Щоб спростити даний вираз, потрібно звести подібні члени. Подібні члени — це ті, що мають однакову змінну з однаковим степенем.\n\n1. Зведемо подібні члени з \( x^2 \):\n \[\n 3x^2 - 2x^2 = 1x^2\n \]\n\n2. Зведемо подібні члени з \( x \):\n \[\n -5x + 4x = -1x\n \]\n\n3. Зведемо постійні члени:\n \[\n 7 - 9 = -2\n \]\n\nОтже, спрощений вираз буде:\n\[\n1x^2 - 1x - 2\n\]\n\nВІДПОВІДЬ: \( x^2 - x - 2 \)"


,field,value
0,topic,Системи рівнянь
1,sample_task,Розв'яжіть систему рівнянь: $\begin{cases} x + y = 10 \\ x - y = 4 \end{cases}$
2,sample_solution,"Використаємо метод додавання. Додамо два рівняння: $(x + y) + (x - y) = 10 + 4$. Отримуємо $2x = 14$, звідки $x = 7$. Підставимо $x$ у перше рівняння: $7 + y = 10$, отже $y = 3$."
3,sample_answer,"$x = 7, y = 3$"
4,system_prompt,Ти - вчитель математики. Створи одну математичну задачу на задану тему та надай її рішення та фінальну відповідь.\nФормат відповіді:\nЗАДАЧА: [текст задачі]\nРІШЕННЯ: [хід рішення]\nВІДПОВІДЬ: [фінальна відповідь]
5,user_message,Системи рівнянь
6,model_response,"ЗАДАЧА: \nРозв'яжіть систему рівнянь:\n1) \(2x + 3y = 12\)\n2) \(4x - y = 5\)\n\nРІШЕННЯ:\nЩоб розв'язати цю систему рівнянь, скористаємося методом підстановки або методом додавання. У цьому випадку використаємо метод додавання.\n\n1) Помножимо друге рівняння на 3, щоб коефіцієнти при \(y\) стали однаковими:\n \[\n 3(4x - y) = 3 \cdot 5 \Rightarrow 12x - 3y = 15\n \]\n\n2) Тепер у нас є дві системи рівнянь:\n \[\n \begin{cases} \n 2x + 3y = 12 \\\n 12x - 3y = 15 \n \end{cases}\n \]\n\n3) Додамо обидва рівняння, щоб позбутися \(y\):\n \[\n (2x + 3y) + (12x - 3y) = 12 + 15\n \]\n \[\n 14x = 27\n \]\n\n4) Знайдемо \(x\):\n \[\n x = \frac{27}{14}\n \]\n\n5) Підставимо знайдене значення \(x\) у перше рівняння, щоб знайти \(y\):\n \[\n 2\left(\frac{27}{14}\right) + 3y = 12\n \]\n \[\n \frac{54}{14} + 3y = 12\n \]\n \[\n 3y = 12 - \frac{54}{14}\n \]\n \[\n 3y = \frac{168}{14} - \frac{54}{14}\n \]\n \[\n 3y = \frac{114}{14}\n \]\n \[\n y = \frac{114}{42}\n \]\n \[\n y = \frac{57}{21}\n \]\n \[\n y = \frac{19}{7}\n \]\n\nВІДПОВІДЬ: \(x = \frac{27}{14}\), \(y = \frac{19}{7}\)"


,field,value
0,topic,Арифметична прогресія
1,sample_task,"Дано арифметичну прогресію, де $a_1 = 5$ і різниця $d = 3$. Знайдіть п'ятий член прогресії ($a_5$)."
2,sample_solution,Використовуємо формулу $n$-го члена арифметичної прогресії: $a_n = a_1 + d(n - 1)$. Підставляємо значення для $n=5$: $a_5 = 5 + 3(5 - 1) = 5 + 3 \cdot 4 = 5 + 12 = 17$.
3,sample_answer,17
4,system_prompt,Ти - вчитель математики. Створи одну математичну задачу на задану тему та надай її рішення та фінальну відповідь.\nФормат відповіді:\nЗАДАЧА: [текст задачі]\nРІШЕННЯ: [хід рішення]\nВІДПОВІДЬ: [фінальна відповідь]
5,user_message,Арифметична прогресія
6,model_response,"ЗАДАЧА: У класі учні вивчають арифметичну прогресію. Вони отримали завдання знайти суму перших 20 членів арифметичної прогресії, якщо перший член прогресії дорівнює 5, а різниця між сусідніми членами прогресії дорівнює 3.\n\nРІШЕННЯ: \nАрифметична прогресія визначається першим членом \( a_1 \) та різницею \( d \). Формула для n-го члена прогресії виглядає так:\n\[ a_n = a_1 + (n-1) \cdot d \]\n\nДля даної прогресії:\n- \( a_1 = 5 \)\n- \( d = 3 \)\n\nФормула для суми перших n членів арифметичної прогресії:\n\[ S_n = \frac{n}{2} \cdot (a_1 + a_n) \]\n\nСпочатку знайдемо 20-й член прогресії \( a_{20} \):\n\[ a_{20} = 5 + (20-1) \cdot 3 = 5 + 57 = 62 \]\n\nТепер знайдемо суму перших 20 членів:\n\[ S_{20} = \frac{20}{2} \cdot (5 + 62) = 10 \cdot 67 = 670 \]\n\nВІДПОВІДЬ: Сума перших 20 членів арифметичної прогресії дорівнює 670."


,field,value
0,topic,Логарифми
1,sample_task,Обчисліть значення логарифма: $\log_2 32$.
2,sample_solution,"Логарифм $\log_a b$ — це показник степеня, до якого треба піднести основу $a$, щоб отримати $b$. Оскільки $2^5 = 32$, то $\log_2 32 = 5$."
3,sample_answer,5
4,system_prompt,Ти - вчитель математики. Створи одну математичну задачу на задану тему та надай її рішення та фінальну відповідь.\nФормат відповіді:\nЗАДАЧА: [текст задачі]\nРІШЕННЯ: [хід рішення]\nВІДПОВІДЬ: [фінальна відповідь]
5,user_message,Логарифми
6,model_response,"ЗАДАЧА: Знайдіть значення виразу \( \log_2(32) + \log_2(8) - \log_2(4) \).\n\nРІШЕННЯ: \nСпочатку розглянемо кожен логарифм окремо:\n\n1. \( \log_2(32) \): \n - Ми знаємо, що \( 32 = 2^5 \).\n - Отже, \( \log_2(32) = \log_2(2^5) = 5 \).\n\n2. \( \log_2(8) \): \n - Ми знаємо, що \( 8 = 2^3 \).\n - Отже, \( \log_2(8) = \log_2(2^3) = 3 \).\n\n3. \( \log_2(4) \): \n - Ми знаємо, що \( 4 = 2^2 \).\n - Отже, \( \log_2(4) = \log_2(2^2) = 2 \).\n\nТепер підставимо ці значення у вираз:\n\n\[ \log_2(32) + \log_2(8) - \log_2(4) = 5 + 3 - 2 \]\n\nОбчислимо:\n\n\[ 5 + 3 - 2 = 6 \]\n\nВІДПОВІДЬ: 6"
